In [12]:
!git clone 'https://github.com/Lorenz92/SKADC1.git'
% cd SKADC1
!echo $PWD

Cloning into 'SKADC1'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 41 (delta 17), reused 25 (delta 6), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import sys
from astropy.io import fits
import os

import src.dataset as dataset
import src.config as config
from src.utils import *
import src.preprocessing as prep

# from src.config import * 

%load_ext autoreload
%autoreload 2

np.random.seed(config.RANDOM_SEED)

In [7]:
for download_info in config.required_files:
    if not os.path.exists(os.path.join(config.TRAIN_DATA_FOLDER, download_info['file_name'])):
        download_data(download_info['file_name'], download_info['url'], config.DOWNLOAD_FOLDER)

Download completed!
Download completed!


In [17]:
# Dataset parsing and loading
# use "subset" to load a small portion of data for development/debugging purposes

ska_dataset = dataset.SKADataset(train_set_path=config.TRAIN_SET_PATH, subset=config.DATA_SUBSET)
print(ska_dataset.raw_train_df.shape)
ska_dataset.raw_train_df.head()

(19222, 15)


,ID,RA (core),DEC (core),RA (centroid),DEC (centroid),FLUX,Core frac,BMAJ,BMIN,PA,SIZE,CLASS,SELECTION,x,y
0,24121695,-0.642227,-29.775242,-0.642316,-29.775211,0.000035,0.000000,5.600,5.551,328.554,1,1,1,19704.519,17712.942
1,20167150,-0.390473,-29.867493,-0.390667,-29.868851,0.029555,0.016924,107.110,43.263,191.258,1,1,1,18401.338,17160.919
2,20045120,-0.021411,-29.643486,-0.021359,-29.643561,0.000653,0.012185,5.037,3.525,276.587,1,1,1,16493.600,18506.577
3,33938384,-0.316375,-29.697647,-0.316375,-29.697647,0.000017,0.000000,0.847,0.266,346.641,2,1,1,18020.319,18182.095
4,7527248,-0.336528,-29.627335,-0.336527,-29.627338,0.000054,0.044763,3.343,1.515,279.006,1,1,1,18125.824,18600.680


In [4]:
data_560Mhz_1000h_fits = fits.open('./data/training/560Mhz_1000h.fits')
# data_560Mhz_1000h_fits.info()
data_560Mhz_1000h_fits.filename()

# print(data_560Mhz_1000h_fits[0].data)
# print(data_560Mhz_1000h_fits[0].data.shape)

data_560Mhz_1000h = data_560Mhz_1000h_fits[0].data[0,0]


In [28]:
# Visualization test
from astropy.io import fits
from astropy.utils.data import download_file

# image_file = download_file('https://owncloud.ia2.inaf.it/index.php/s/hbasFhd4YILNkCr/download', cache=True )

# image_data = fits.getdata(image_file)
# image_data = image_data.reshape(image_data.shape[2:])

# compute per value to clip the image
perc = 99.9
percentileThresh = np.percentile(data_560Mhz_1000h, perc)
print (f'{perc} percentile {percentileThresh}')

# # crop the image to speed up computation and clip using per value
# image_crop= image_data[5000:10000, 5000:10000]
# image_crop.clip(max=percentileThresh)

# plt.figure(figsize=(100, 100))
# # plt.imshow(normalized_data, cmap='viridis', vmax=255, vmin=0)
# for idx in range(25):
#     plt.subplot(5, 5, idx+1)
#     plt.axis('off')
#     base = 200 * idx
#     end = base + 200
#     normalized_data = image_crop[base:end, base:end] * (1.0 / percentileThresh)
#     plt.imshow(normalized_data, cmap='viridis', vmax=1, vmin=0)

99.9 percentile 8.316711500810925e-06


### Preprocessing

In [18]:
coords = ska_dataset._convert_boxes_to_px_coord(ska_dataset.raw_train_df, data_560Mhz_1000h_fits[0].header)

In [14]:
pd.DataFrame.from_dict(coords)

,x1,y1,x2,y2,major_semia_px,minor_semia_px,pa_in_rad,width,height
0,19511.716409,18336.438393,19512.236701,18337.368476,0.520481,0.114191,-1.092454,0.520292,0.930083
1,17462.766064,18237.777763,17463.578359,18238.502373,0.521308,0.156392,-0.717069,0.812295,0.724610
2,18743.873128,18203.983828,18744.619426,18205.158678,0.640464,0.272239,1.114969,0.746297,1.174850
3,18243.029932,19392.024143,18243.201470,19392.677135,0.326852,0.084402,-1.522486,0.171537,0.652992
4,18096.592661,16771.274132,18097.137239,16772.532130,0.640464,0.244105,-1.365615,0.544578,1.257999
...,...,...,...,...,...,...,...,...,...
1917,17282.263190,18813.248637,17282.766523,18814.383517,0.591643,0.187836,-1.267633,0.503333,1.134880
1918,18320.371448,18524.427677,18321.447576,18525.365962,0.679355,0.219280,-0.701046,1.076128,0.938285
1919,16586.905882,17846.493424,16590.199428,17851.852471,3.075718,0.657014,1.044248,3.293546,5.359047
1920,17376.842669,17542.989490,17377.028023,17543.174844,0.092677,0.092677,3.641996,0.185354,0.185354


In [19]:
df_train = ska_dataset._extend_dataframe(ska_dataset.raw_train_df, coords)
df_train.head()

,ID,RA (core),DEC (core),RA (centroid),DEC (centroid),FLUX,Core frac,BMAJ,BMIN,PA,...,y,x1,y1,x2,y2,major_semia_px,minor_semia_px,pa_in_rad,width,height
0,24121695,-0.642227,-29.775242,-0.642316,-29.775211,0.000035,0.000000,5.600,5.551,328.554,...,17712.942,19699.889820,17708.334874,19709.135520,17717.543623,4.633850,4.593304,5.734349,9.245699,9.208748
1,20167150,-0.390473,-29.867493,-0.390667,-29.868851,0.029555,0.016924,107.110,43.263,191.258,...,17160.919,18314.128617,17121.775102,18488.540079,17200.059688,88.630651,35.798972,3.338082,174.411463,78.284586
2,20045120,-0.021411,-29.643486,-0.021359,-29.643561,0.000653,0.012185,5.037,3.525,276.587,...,18506.577,16490.663037,18502.418816,16496.536575,18510.726749,4.167982,2.916843,4.827354,5.873538,8.307933
3,33938384,-0.316375,-29.697647,-0.316375,-29.697647,0.000017,0.000000,0.847,0.266,346.641,...,18182.095,18019.631633,18181.823147,18020.999232,18182.360119,0.700870,0.220108,6.050027,1.367598,0.536972
4,7527248,-0.336528,-29.627335,-0.336527,-29.627338,0.000054,0.044763,3.343,1.515,279.006,...,18600.680,18124.509296,18597.936034,18127.132703,18603.414392,2.766243,1.253622,4.869573,2.623407,5.478358


In [8]:
x1_min = int(np.floor(min(df_train['x1'])))
y1_min = int(np.floor(min(df_train['y1'])))

x2_max = int(np.floor(max(df_train['x2'])))
y2_max = int(np.floor(max(df_train['y2'])))

data_560Mhz_1000h_train = data_560Mhz_1000h[y1_min:y2_max, x1_min:x2_max]
data_560Mhz_1000h_train.shape


(3177, 3451)

In [9]:
fits_max_data_560Mhz_1000h = np.max(data_560Mhz_1000h)
print(fits_max_data_560Mhz_1000h)

data_560Mhz_1000h_train_clipped = np.clip(data_560Mhz_1000h_train, a_min=0, a_max=fits_max_data_560Mhz_1000h)

0.48094657


In [27]:
data_560Mhz_1000h_train_clipped

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 5.1189630e-07,
        1.1818922e-07, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 3.5396201e-07,
        7.6755441e-08, 0.0000000e+00],
       [1.0576650e-07, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.4718134e-07,
        3.5016120e-07, 2.1914468e-07],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.6405086e-07,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 2.6262634e-07,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [37]:
patches={}
patches = ska_dataset._split_in_patch(data_560Mhz_1000h_train_clipped, df_train, data_560Mhz_1000h_fits.filename(), x1_min, y1_min, 205) # Remember to remove internal limit

image saved
Index(['ID', 'RA (core)', 'DEC (core)', 'RA (centroid)', 'DEC (centroid)',
       'FLUX', 'Core frac', 'BMAJ', 'BMIN', 'PA', 'SIZE', 'CLASS', 'SELECTION',
       'x', 'y', 'x1', 'y1', 'x2', 'y2', 'major_semia_px', 'minor_semia_px',
       'pa_in_rad', 'width', 'height', 'x1s', 'y1s', 'x2s', 'y2s',
       'patch_name', 'patch_xo', 'patch_yo', 'patch_dim'],
      dtype='object')
           ID  RA (core)  DEC (core)  RA (centroid)  DEC (centroid)      FLUX  \
11   27514971  -0.014459  -29.933004      -0.014121      -29.932705  0.039324   
107  14022077  -0.039882  -29.915529      -0.039872      -29.915523  0.000157   
115  15252327  -0.039005  -29.915081      -0.039002      -29.915081  0.000010   
156  13302538  -0.005224  -29.939449      -0.005224      -29.939449  0.000017   
427  30994188  -0.035645  -29.910868      -0.035645      -29.910868  0.000934   

     Core frac    BMAJ    BMIN       PA  ...      width     height  \
11    0.004216  31.508  17.616  353.308  ...  51.90

In [33]:
a = np.load(os.path.join(config.TRAIN_PATCHES_FOLDER, 'your_file.npy'))
print(a)

[[0.0000000e+00 5.0277004e-07 8.1352823e-07 ... 2.1992520e-07
  3.3757752e-07 8.9618879e-08]
 [0.0000000e+00 4.7234460e-07 7.5504875e-07 ... 0.0000000e+00
  2.7678368e-08 3.8842813e-08]
 [0.0000000e+00 0.0000000e+00 4.7753889e-07 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [1.6627402e-07 2.2166087e-07 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.3635220e-07 2.9386493e-07 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]


#TODO:
- Ingrandire bbox
- unire bbox sovrapposte
- Mergiare branch data_clean (anche alla fine)
- Quando vengono create le patch, ritagliare le bbox per farle stare tutte nella patch X
- Check tra nostro dataset pulito e quello di ICRAR (dopo aver mergiato il branch data_clean)

Da capire per la rete:
- bisogna scalre input tra 0 e 255 intero?
- salvare le patch come immagini o possiamo darle in input on the fly?
- è necessario un custom input layer che converte le nostra immagini da 1 a 3 canali?

In [5]:
# Download VGG16 weights
# 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

download_data('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 
'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', config.MODEL_WEIGHTS + '/vgg16')


Download completed!


In [8]:
# First net version
import keras
from keras import backend as K
from keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Concatenate

FROZEN_LAYERS_IDX = [1, 2, 4, 5]
ANCHOR_NUM = 6
PROB_PRED_OUT = 2 * ANCHOR_NUM
COORD_PRED_OUT = 4 * ANCHOR_NUM 
input_shape=(600, 600, 1) #TODO: change to 205,205,1

vgg16_weight_path = os.path.join('./model/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

# Add custom input-layer to change from1 to 3 channels
inp = Input(input_shape)
expander_layer = Conv2D(filters=3, kernel_size=(1, 1), padding='same', activation='relu', name='Custom_input_layer')(inp)

'''
# Load pretrained VGG16 and remove last MaxPool layer
# VGG16 = keras.applications.VGG16(include_top=False, weights="imagenet", pooling=None) # Commentato perchè non siamo riusciti a togliere l'ultimo layer
# x = x(expander_layer)
'''

# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', trainable=False)(expander_layer)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', trainable=False)(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', trainable=False)(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', trainable=False)(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
# x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

print(x)


# # Freeze first 4 convolutional layers
# for idx in FROZEN_LAYERS_IDX:
#   VGG16.layers[idx].trainable = False

# # Create Region Proposal Net
x = Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu', name='18_RPN_Conv1')(x)
cls_pred = Conv2D(filters=PROB_PRED_OUT, kernel_size=(1, 1), padding='same', activation='sigmoid', name='19_Anchor_Cls_Conv')(x) #output of layer 20
reg_pred = Conv2D(filters=COORD_PRED_OUT, kernel_size=(1, 1), padding='same', activation='linear', name='19_Anchor_Reg_Conv')(x) #output of layer 19

print(x)
print(cls_pred.shape)
print(reg_pred.shape)

anch_target_cls = Reshape((cls_pred.shape[1]**2, cls_pred.shape[3]), name='20_Anchor_Target_cls')(cls_pred)
anch_target_reg = Reshape((reg_pred.shape[1]**2, reg_pred.shape[3]), name='20_Anchor_Target_reg')(reg_pred)

print(anch_target_cls)
print(anch_target_reg)

region_prop_net = keras.Model(inp, [cls_pred, reg_pred], name='Region Proposal')

region_prop_net.load_weights(vgg16_weight_path, by_name=True)

# region_prop_net.compile(loss='categorical_crossentropy', optimizer='adam')

region_prop_net.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 37, 37, 512), dtype=tf.float32, name=None), name='block5_conv3/Relu:0', description="created by layer 'block5_conv3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 37, 37, 512), dtype=tf.float32, name=None), name='18_RPN_Conv1/Relu:0', description="created by layer '18_RPN_Conv1'")
(None, 37, 37, 12)
(None, 37, 37, 24)
KerasTensor(type_spec=TensorSpec(shape=(None, 1369, 12), dtype=tf.float32, name=None), name='20_Anchor_Target_cls/Reshape:0', description="created by layer '20_Anchor_Target_cls'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1369, 24), dtype=tf.float32, name=None), name='20_Anchor_Target_reg/Reshape:0', description="created by layer '20_Anchor_Target_reg'")
Model: "Region Proposal"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 600, 60

In [ ]:
def rpn_loss_regr(num_anchors):
    """Loss function for rpn regression
    Args:
        num_anchors: number of anchors (9 in here)
    Returns:
        Smooth L1 loss function 
                           0.5*x*x (if x_abs < 1)
                           x_abx - 0.5 (otherwise)
    """
    def rpn_loss_regr_fixed_num(y_true, y_pred):

        # x is the difference between true value and predicted value
        x = y_true[:, :, :, 4 * num_anchors:] - y_pred

        # absolute value of x
        x_abs = K.abs(x)

        # If x_abs <= 1.0, x_bool = 1
        x_bool = K.cast(K.less_equal(x_abs, 1.0), tf.float32)

        return lambda_rpn_regr * K.sum(
            y_true[:, :, :, :4 * num_anchors] * (x_bool * (0.5 * x * x) + (1 - x_bool) * (x_abs - 0.5))) / K.sum(epsilon + y_true[:, :, :, :4 * num_anchors])

    return rpn_loss_regr_fixed_num

In [115]:
y_rpn_regr = np.zeros((3, 3, 3 * 4))
y_rpn_regr[0] = 9
y_rpn_regr[1] = 1
y_rpn_regr

array([[[9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
        [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.],
        [9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.]],

       [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [116]:
y_rpn_regr = np.transpose(y_rpn_regr, (2, 0, 1))
print(y_rpn_regr)

[[[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[9. 9. 9.]
  [1. 1. 1.]
  [0. 0. 0.]]]


In [117]:
y_rpn_regr = np.expand_dims(y_rpn_regr, axis=0)
print(y_rpn_regr)

[[[[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]

  [[9. 9. 9.]
   [1. 1. 1.]
   [0. 0. 0.]]]]


In [14]:
y_rpn_regr.shape

NameError: name 'y_rpn_regr' is not defined

In [ ]:
a,b,c = prep.calc_rpn(df_train.head(1), 600, 600,600, 600)

Error: connect ECONNREFUSED 127.0.0.1:63301

TODO - 20210502:
- creare colonne per coordinate bbox nel sistema di riferimento della patch
- creare cartelle per patches con jpeg + pkl delle sue sole bbox
- ragionare su dimensioni patch
- X Caricare pesi VGG16 nel nuovo network
- lanciare training

In [ ]:
#TODO: Provare YOLO che ad Asperti piace
# Ablation study
# Provare, a parità di frequenza, le esposizioni diverse come canali